### Задача (варіант 2)

**Ієрархія:** p = 3, повна, m = (3, 3, 4)

**Метод розрахунку глобальних ваг:** ідеальної точки, ГВБВПА

*Методи аналізу альтернатив рішень на основі ієрархічної моделі критеріїв (МАІ, analytic hierarchy process, AHP)* складаються з наступних чотирьох загальних етапів:

1. Побудова ієрархічної моделі критеріїв, цілей та інших факторів, які впливають на головну ціль прийняття рішення; побудова множини альтернативних варіантів рішень.

2. Отримання суджень експертів щодо парних порівнянь елементів одного рівня ієрархії відносно спільного елементу вищого рівня. Парні порівняння проводяться у вибраній шкалі і за результатами будуються матриці парних порівнянь (МПП), які є обернено симетричними.

3. Математична обробка суджень експертів:
   * розрахунок локальних ваг елементів кожного рівня ієрархії відповідно до батьківських елементів вищого рівня на основі МПП; побудова локальних ранжувань;
   * аналіз узгодженості експертних оцінок;
   * розрахунок глобальних ваг елементів ієрархії відносно головної цілі прийняття рішення, використовуючи методи агрегування; побудова ранжування на основі глобальних ваг.
   * вилучити з розгляду найменш вагомий критерій і виконати етапи
   * по черзі додати декілька альтернатив-копій еквівалентних до однієї з існуючих альтернатив, в кожному з експериментів виконати етапи  для множини з N+1 альтернатив,
   * по черзі додати декілька неоптимальних за усіма критеріями альтернатив, в кожному з експериментів виконати етапи для множини з N+1 альтернатив,
   * додати альтернативу, оптимальну за одним з критеріїв, в кожному з експериментів виконати етапи для множини з N+1 альтернатив,

4. Порівняти ваги, отримані різними методами агрегування, та оцінити чутливість результуючого ранжування до зміни множин критеріїв та альтернатив рішень

Постановка задачі:

**Дано:** 
 * $A = \{A_i | i = 1, \dots, N \}$ - множина альтернативних варіантів рішень;
 * $C = \{C_j | j = 1, \dots, M \}$ - множина критеріїв оцінювання альтернатив;
 * $a_{ij}$ - ненормована вага льтернативи $A_i$ за критерієм $C_j$;
 * $w_j^C$ - вага критерію $C_j, \sum_{j=1}^{M} w_j^C = 1$

**Потрібно:**
 * знайти глобальні ваги $w_i^{глоб}$ альтернатив $A_i, i = 1, \dots, N$

In [1]:
import numpy as np
import math

In [2]:
# number of alternatives
n = 4
# number of criterions C
m = [3, 3, 4]

#number of layers = P
p = 3

Функція для генерування коректної матриці парних порівнянь для мультиплікативної групи

In [3]:
def generate_matched_matrix(n, low=1, high=10):
    """
    Generates matched multiplicational matrix.
    """
    mmp = np.zeros((n, n))
    temp = np.random.randint(low, high, size=n).astype(float)
    powers = np.random.randint(-1, 2, size=n).astype(float)
    mmp[0, :] = np.power(temp, powers)
    j = 0
    for i in range(1, n):
        mmp[i, j] = 1 / mmp[j, i]
    for i in range(1, n):
        for j in range(1, n):
            mmp[i, j] = mmp[i, 0] * mmp[0, j]
    mmp[0][0] = 1
    return mmp

# Data generation

Генеруємо матриці парних порівнянь, та ваги критеріїв.

In [4]:
# put some fixed seed for reproducibility of the results
np.random.seed(44)
# matrices of pair comparisons according to each of 'm' criterion
ddd = [[generate_matched_matrix(m[0])]]
for layer in range(p - 1):
    dd = []
    for i in range(m[layer]):
        dd.append(generate_matched_matrix(m[layer + 1]))
    ddd.append(dd)

# weights of criterions
#w_crit = np.random.randn(m, 1)
ddd

[[array([[1.  , 0.25, 0.5 ],
         [4.  , 1.  , 2.  ],
         [2.  , 0.5 , 1.  ]])], [array([[1.   , 1.   , 8.   ],
         [1.   , 1.   , 8.   ],
         [0.125, 0.125, 1.   ]]), array([[1.        , 1.        , 6.        ],
         [1.        , 1.        , 6.        ],
         [0.16666667, 0.16666667, 1.        ]]), array([[1. , 1. , 0.2],
         [1. , 1. , 0.2],
         [5. , 5. , 1. ]])], [array([[1.        , 3.        , 9.        , 9.        ],
         [0.33333333, 1.        , 3.        , 3.        ],
         [0.11111111, 0.33333333, 1.        , 1.        ],
         [0.11111111, 0.33333333, 1.        , 1.        ]]),
  array([[ 1.  ,  4.  ,  0.2 ,  1.  ],
         [ 0.25,  1.  ,  0.05,  0.25],
         [ 5.  , 20.  ,  1.  ,  5.  ],
         [ 1.  ,  4.  ,  0.2 ,  1.  ]]),
  array([[1.        , 0.33333333, 1.        , 0.11111111],
         [3.        , 1.        , 3.        , 0.33333333],
         [1.        , 0.33333333, 1.        , 0.11111111],
         [9.        ,

Обраховуємо локальні ваги використовуючи підхід EM з попередньої лабораторної роботи

In [5]:
def calculate_local(D):
    weighcalc = lambda x: math.pow(np.prod(x),1/4.)
    
    res = []
    for layer in D:
        layerres = []
        for d in layer:
            v = [weighcalc(di) for di in d]
            w = [vi/np.sum(v) for vi in v]
            layerres.append(w)
        res.append(layerres)
    return res
v = calculate_local(ddd)
v

[[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183]],
 [[0.4524428080730101, 0.4524428080730101, 0.09511438385397966],
  [0.44231202279518583, 0.44231202279518583, 0.11537595440962829],
  [0.18713620050514732, 0.18713620050514732, 0.6257275989897054]],
 [[0.642857142857143,
   0.21428571428571433,
   0.07142857142857144,
   0.07142857142857144],
  [0.13793103448275862,
   0.034482758620689655,
   0.6896551724137931,
   0.13793103448275862],
  [0.07142857142857142,
   0.21428571428571425,
   0.07142857142857142,
   0.6428571428571428]]]

# Methods for finding global normed weights of alternatives

Знайдемо локальні ваги альтернатив рішень та ваги критеріїв за
методом ідеальної точки

In [6]:
def ideal_point_synthesis(v):
    """
    Computes global weights
    """
    w_glob = [v[0][0]]
    for i in range(1, p):
        r = [np.divide(np.array(vi), np.array(vi).max()) for vi in v[i]]
        vi = [np.sum(vic) for vic in np.multiply(np.array(r).T, w_glob[i - 1])]
        wi_glob = [vij/np.sum(vi) for vij in vi]
        w_glob.append(wi_glob)
    return w_glob
w_g = ideal_point_synthesis(v)
w_g

[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183],
 [0.38356076046023996, 0.38356076046023996, 0.23287847907952017],
 [0.32087543988563444,
  0.1482822537233742,
  0.29837197692374073,
  0.23247032946725066]]

Знайдемо локальні ваги альтернатив рішень та ваги критеріїв за
методом групового врахування бінарних відношень переваг з ЕМ з попередньої роботи

In [7]:
def groupped_concideration_of_binary_alternative_superiority_relation(aij, wjc):
    #gvbvpa
    n = len(aij[0])
    P = np.identity(n)
    
    for i in range(n):
        for k in range(i, n):
            ri = [aj[i] / (aj[i] + aj[k]) for aj in aij]
            rk = [aj[k] / (aj[i] + aj[k]) for aj in aij]
            wi = np.sum(np.multiply(ri,wjc))
            wk = np.sum(np.multiply(rk,wjc))
            P[i, k] = wi / wk
            P[k, i] = wk / wi
    
    weighcalc = lambda x: math.pow(np.prod(x),1/4.)
    v = [weighcalc(di) for di in P]
    w = [vi / np.sum(v) for vi in v]
    return w

_aij = v[-1]
_wjc = w_g[-2]
w_gvbvpa = groupped_concideration_of_binary_alternative_superiority_relation(_aij, _wjc)
w_gvbvpa

[0.3141295140757584,
 0.19069104795749503,
 0.2532685763609987,
 0.24191086160574785]

In [8]:
print("Найкраща альтернатива за методом ідеальної точки " + str(np.argmax(w_g[-1]) + 1))
print("Найкраща альтернатива за методом ГВБВПА " + str(np.argmax(w_gvbvpa) + 1))

Найкраща альтернатива за методом ідеальної точки 1
Найкраща альтернатива за методом ГВБВПА 1


# Аналіз чутливості
### Вилучимо з розгляду найменш вагомий критерій

In [9]:
# removing a criterion (upper level)
layer = 0
least_criterion = np.argmin(w_g[layer])
sans_lc = list(range(m[layer]))
sans_lc.pop(least_criterion)
ddd_modified = ddd[:layer]
ddd_modified.append([dd[:,sans_lc][sans_lc,:] for dd in ddd[layer]])
ddd_modified.append([ddd[layer + 1][i] for i in sans_lc])
ddd_modified.extend(ddd[layer + 2:])
ddd_modified

[[array([[1. , 2. ],
         [0.5, 1. ]])], [array([[1.        , 1.        , 6.        ],
         [1.        , 1.        , 6.        ],
         [0.16666667, 0.16666667, 1.        ]]), array([[1. , 1. , 0.2],
         [1. , 1. , 0.2],
         [5. , 5. , 1. ]])], [array([[1.        , 3.        , 9.        , 9.        ],
         [0.33333333, 1.        , 3.        , 3.        ],
         [0.11111111, 0.33333333, 1.        , 1.        ],
         [0.11111111, 0.33333333, 1.        , 1.        ]]),
  array([[ 1.  ,  4.  ,  0.2 ,  1.  ],
         [ 0.25,  1.  ,  0.05,  0.25],
         [ 5.  , 20.  ,  1.  ,  5.  ],
         [ 1.  ,  4.  ,  0.2 ,  1.  ]]),
  array([[1.        , 0.33333333, 1.        , 0.11111111],
         [3.        , 1.        , 3.        , 0.33333333],
         [1.        , 0.33333333, 1.        , 0.11111111],
         [9.        , 3.        , 9.        , 1.        ]])]]

Обчислимо ваги методами ідеальної точки і ГВБВПА

In [10]:
v_dash = calculate_local(ddd_modified)
w_g_dash = ideal_point_synthesis(v_dash)
_aij = v_dash[-1]
_wjc = w_g_dash[-2]
w_gvbvpa_dash = groupped_concideration_of_binary_alternative_superiority_relation(_aij, _wjc)

In [11]:
print("Ваги отримані методом ідеальної точки")
w_g_dash

Ваги отримані методом ідеальної точки


[[0.585786437626905, 0.41421356237309503],
 [0.35727189894488437, 0.35727189894488437, 0.2854562021102313],
 [0.3033538226552575,
  0.1529184609405944,
  0.2824310452590522,
  0.26129667114509597]]

In [12]:
print("Ваги альтернатив отримані методом ГВБВПА")
w_gvbvpa_dash

Ваги альтернатив отримані методом ГВБВПА


[0.29636918915545996,
 0.19877541153539638,
 0.24302735269481468,
 0.261828046614329]

In [13]:
print("Найкраща альтернатива за методом ідеальної точки " + str(np.argmax(w_g_dash[-1]) + 1))
print("Найкраща альтернатива за методом ГВБВПА " + str(np.argmax(w_gvbvpa_dash) + 1))

Найкраща альтернатива за методом ідеальної точки 1
Найкраща альтернатива за методом ГВБВПА 1


### Продублюємо довільну альтернативу

In [14]:
alternative = np.random.randint(m[-1])
duplicated = list(range(m[-1]))
duplicated.insert(alternative, alternative)
ddd_modified = ddd[:-1]
ddd_modified.append([dd[:, duplicated][duplicated, :] for dd in ddd[-1]])
print("Довільним чином обрана альтернатива " + str(alternative + 1))
ddd_modified[-1]

Довільним чином обрана альтернатива 3


[array([[1.        , 3.        , 9.        , 9.        , 9.        ],
        [0.33333333, 1.        , 3.        , 3.        , 3.        ],
        [0.11111111, 0.33333333, 1.        , 1.        , 1.        ],
        [0.11111111, 0.33333333, 1.        , 1.        , 1.        ],
        [0.11111111, 0.33333333, 1.        , 1.        , 1.        ]]),
 array([[ 1.  ,  4.  ,  0.2 ,  0.2 ,  1.  ],
        [ 0.25,  1.  ,  0.05,  0.05,  0.25],
        [ 5.  , 20.  ,  1.  ,  1.  ,  5.  ],
        [ 5.  , 20.  ,  1.  ,  1.  ,  5.  ],
        [ 1.  ,  4.  ,  0.2 ,  0.2 ,  1.  ]]),
 array([[1.        , 0.33333333, 1.        , 1.        , 0.11111111],
        [3.        , 1.        , 3.        , 3.        , 0.33333333],
        [1.        , 0.33333333, 1.        , 1.        , 0.11111111],
        [1.        , 0.33333333, 1.        , 1.        , 0.11111111],
        [9.        , 3.        , 9.        , 9.        , 1.        ]])]

In [15]:
v_dash = calculate_local(ddd_modified)
w_g_dash = ideal_point_synthesis(v_dash)
_aij = v_dash[-1]
_wjc = w_g_dash[-2]
w_gvbvpa_dash = groupped_concideration_of_binary_alternative_superiority_relation(_aij, _wjc)
print("Ваги отримані методом ідеальної точки")
w_g_dash

Ваги отримані методом ідеальної точки


[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183],
 [0.38356076046023996, 0.38356076046023996, 0.23287847907952017],
 [0.2541252341525689,
  0.09333335129132692,
  0.23904323019669635,
  0.23904323019669635,
  0.17445495416271156]]

In [16]:
print("Ваги альтернатив отримані методом ГВБВПА")
w_gvbvpa_dash

Ваги альтернатив отримані методом ГВБВПА


[0.26491896416919325,
 0.1491552860804503,
 0.2008431231490342,
 0.2008431231490342,
 0.18423950345228804]

In [17]:
print("Найкраща альтернатива за методом ідеальної точки " + str(np.argmax(w_g_dash[-1]) + 1))
print("Найкраща альтернатива за методом ГВБВПА " + str(np.argmax(w_gvbvpa_dash) + 1))

Найкраща альтернатива за методом ідеальної точки 1
Найкраща альтернатива за методом ГВБВПА 1


### Додавання неоптимальної альтернативи

In [18]:
def insert_worse_alternative(d, low=1, high=10):
    new_size = m[-1] + 1
    new_mmp = np.zeros((new_size, new_size))
    temp = np.random.randint(low, high, size=new_size).astype(float)
    new_mmp[:-1, :-1] = d
    new_mmp[:, -1] = temp
    new_mmp[-1, :] = np.power(temp, -1.)
    new_mmp[-1, -1] = 1.
    return new_mmp

ddd_modified = ddd[:-1]
ddd_modified.append([insert_worse_alternative(dd) for dd in ddd[-1]])
ddd_modified[-1]

[array([[1.        , 3.        , 9.        , 9.        , 5.        ],
        [0.33333333, 1.        , 3.        , 3.        , 7.        ],
        [0.11111111, 0.33333333, 1.        , 1.        , 8.        ],
        [0.11111111, 0.33333333, 1.        , 1.        , 6.        ],
        [0.2       , 0.14285714, 0.125     , 0.16666667, 1.        ]]),
 array([[ 1.        ,  4.        ,  0.2       ,  1.        ,  7.        ],
        [ 0.25      ,  1.        ,  0.05      ,  0.25      ,  3.        ],
        [ 5.        , 20.        ,  1.        ,  5.        ,  6.        ],
        [ 1.        ,  4.        ,  0.2       ,  1.        ,  2.        ],
        [ 0.14285714,  0.33333333,  0.16666667,  0.5       ,  1.        ]]),
 array([[1.        , 0.33333333, 1.        , 0.11111111, 2.        ],
        [3.        , 1.        , 3.        , 0.33333333, 9.        ],
        [1.        , 0.33333333, 1.        , 0.11111111, 1.        ],
        [9.        , 3.        , 9.        , 1.        , 4.  

In [19]:
v_dash = calculate_local(ddd_modified)
w_g_dash = ideal_point_synthesis(v_dash)
_aij = v_dash[-1]
_wjc = w_g_dash[-2]
w_gvbvpa_dash = groupped_concideration_of_binary_alternative_superiority_relation(_aij, _wjc)
print("Ваги отримані методом ідеальної точки")
w_g_dash

Ваги отримані методом ідеальної точки


[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183],
 [0.38356076046023996, 0.38356076046023996, 0.23287847907952017],
 [0.31125521003085144,
  0.1606007635907964,
  0.2886310325991901,
  0.21546626113846307,
  0.024046732640699085]]

In [20]:
print("Ваги альтернатив отримані методом ГВБВПА")
w_gvbvpa_dash

Ваги альтернатив отримані методом ГВБВПА


[0.32080202623264054,
 0.17685968899194765,
 0.23905295858153663,
 0.22834621786687231,
 0.0349391083270029]

In [21]:
print("Найкраща альтернатива за методом ідеальної точки " + str(np.argmax(w_g_dash[-1]) + 1))
print("Найкраща альтернатива за методом ГВБВПА " + str(np.argmax(w_gvbvpa_dash) + 1))

Найкраща альтернатива за методом ідеальної точки 1
Найкраща альтернатива за методом ГВБВПА 1


### Додання альтернативи оптимальної за одним з критеріїв

In [22]:
def insert_better_alternative(d, high=9):
    new_size = m[-1] + 1
    new_mmp = np.zeros((new_size, new_size))
    temp = np.full(new_size, high).astype(float)
    new_mmp[:-1, :-1] = d
    new_mmp[-1, :] = temp
    new_mmp[:, -1] = np.power(temp, -1.)
    new_mmp[-1, -1] = 1.
    return new_mmp

random_criterion = np.random.randint(m[-2])
ddd_modified = ddd[:-1]
ddd_modified.append([insert_worse_alternative(dd) for dd in ddd[-1][:random_criterion]])
ddd_modified[-1].append(insert_better_alternative(ddd[-1][random_criterion]))
ddd_modified[-1].extend([insert_worse_alternative(dd) for dd in ddd[-1][random_criterion + 1:]])
ddd_modified[-1:]

[[array([[1.        , 3.        , 9.        , 9.        , 6.        ],
         [0.33333333, 1.        , 3.        , 3.        , 1.        ],
         [0.11111111, 0.33333333, 1.        , 1.        , 9.        ],
         [0.11111111, 0.33333333, 1.        , 1.        , 1.        ],
         [0.16666667, 1.        , 0.11111111, 1.        , 1.        ]]),
  array([[ 1.        ,  4.        ,  0.2       ,  1.        ,  0.11111111],
         [ 0.25      ,  1.        ,  0.05      ,  0.25      ,  0.11111111],
         [ 5.        , 20.        ,  1.        ,  5.        ,  0.11111111],
         [ 1.        ,  4.        ,  0.2       ,  1.        ,  0.11111111],
         [ 9.        ,  9.        ,  9.        ,  9.        ,  1.        ]]),
  array([[1.        , 0.33333333, 1.        , 0.11111111, 3.        ],
         [3.        , 1.        , 3.        , 0.33333333, 8.        ],
         [1.        , 0.33333333, 1.        , 0.11111111, 7.        ],
         [9.        , 3.        , 9.        , 1.

In [23]:
v_dash = calculate_local(ddd_modified)
w_g_dash = ideal_point_synthesis(v_dash)
_aij = v_dash[-1]
_wjc = w_g_dash[-2]
w_gvbvpa_dash = groupped_concideration_of_binary_alternative_superiority_relation(_aij, _wjc)
print("Ваги отримані методом ідеальної точки")
w_g_dash

Ваги отримані методом ідеальної точки


[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183],
 [0.38356076046023996, 0.38356076046023996, 0.23287847907952017],
 [0.28884463676319605,
  0.11328472540000803,
  0.12788450371382862,
  0.1913484659311749,
  0.2786376681917924]]

In [24]:
print("Ваги альтернатив отримані методом ГВБВПА")
w_gvbvpa_dash

Ваги альтернатив отримані методом ГВБВПА


[0.2691881947477901,
 0.1451193420911868,
 0.2326042285580377,
 0.1725725463794374,
 0.180515688223548]

In [25]:
print("Найкраща альтернатива за методом ідеальної точки " + str(np.argmax(w_g_dash[-1]) + 1))
print("Найкраща альтернатива за методом ГВБВПА " + str(np.argmax(w_gvbvpa_dash) + 1))

Найкраща альтернатива за методом ідеальної точки 1
Найкраща альтернатива за методом ГВБВПА 1
